# Directionality analysis

## Most important DataFrames and parameters
DataFrames:
* all_onsets_df: contains all neurons of all recordings, with onset and quantile values for each event (sz: quantile_sz, sd1: quantile1, sd2: quantile2)
* quantiles_df: for each recording, contains all quantiles for each event type (quantile_type)
* qdf: contains vectors (dx, dy) from first to last quantile centre for each event in all recordings
* df_mean_vectors_per_mouse: contains Cartesian and polar coordinates of vectors for each event type, per mouse, averaged over each recording. Averaging: for one mouse, calculate center of first and last quantiles, create vector as difference; take the average of x and y coordinates of these vectors.
* df_angles: contains seizure-sdx angles


Columns:
* quantile_type: sz, sd1 or sd2
* onset_sz, onset1, onset2: the onset frame for each neuron/quantile for sz, sd1, sd2, respectively
* dx, dy: The distance between the first and last quantile of <quantile_type> in pixels along the x or y axis, respectively.
* uuid_extended: uuid if the recording contained one seizure; uuid + "_1"/"_2"/... for each event group (sz + sd waves) in that recording.
* uuid_matched: manually correct uuid_extended to group seizure with sd waves that were split up by end of recording. This uuid should be unique among event groups (sz + sd waves), and the same for each group member.
* theta_inj_top: the polar angle in a reference frame where the top side is the injection side.

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import h5py
from nd2_to_caiman import np_arr_from_nd2
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from math import floor, sqrt
from datetime import datetime
import json
from labrotation import json_util
import scipy
from scipy import ndimage
import datadoc_util
from statsmodels.nonparametric.smoothers_lowess import lowess
import pandas as pd
import seaborn as sns
from math import floor
import multiprocess as mp  # multiprocessing does not work with IPython. Use fork instead.
import os
import random  # for possible stochastic algorithms

In [ ]:
grid_shape = (8,8)

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

## Open files and get uuid

In [ ]:
df_id_uuid = ddoc.getIdUuid()

In [ ]:
analysis_folder = fh.open_dir("Open directory with analysis (grid) data for all mice!")

In [ ]:
grid_files_list = []
for root, dirs, files in os.walk(analysis_folder):
    for fname in files:
        if "_grid.h5" in fname:
            grid_files_list.append(os.path.join(root,fname))

In [ ]:
uuid_dict = dict()
for grid_fpath in grid_files_list:
    # ..._grid.h5 -> ..._cnmf.hdf5
    cnmf_fpath = os.path.join(os.path.split(grid_fpath)[0], os.path.split(grid_fpath)[-1][:-7] + "cnmf.hdf5")
    with h5py.File(cnmf_fpath, 'r') as hf:
        uuid_dict[grid_fpath] = hf.attrs["uuid"]

## Combine all results into one dataframe

In [ ]:
cols_set = set()
for fpath in grid_files_list:
    df = pd.read_hdf(fpath)
    for key in df.keys():
        cols_set.add(key)
cols_set.add("uuid")
cols_set.add("mouse_id")

In [ ]:
# defining empty dataframe does not work, as all data types will be object (except x, y, which will be proper integers)
all_onsets_df = pd.read_hdf(grid_files_list[0])
for fpath in grid_files_list[1:]:
    df = pd.read_hdf(fpath)
    df["uuid"] = uuid_dict[fpath]
    df["mouse_id"] = df_id_uuid[df_id_uuid["uuid"] == uuid_dict[fpath]]["mouse_id"].values[0]
    assert df["uuid"].isna().sum() == 0
    all_onsets_df = pd.concat([all_onsets_df, df])

In [ ]:
# for old files containing onset data, n_seizures was not present, as one of the last recordings processed contained 2. 
# As a result, most of i_sz values are NaN; these contain 1 sz. Otherwise 0, 1... are the seizure indices.
all_onsets_df["i_sz"].unique()

In [ ]:
# make seizures unique in uuid_extended
def append_uuid(row):
    if pd.isna(row['i_sz']):
        return row['uuid']
    elif row['i_sz'] >= 0:
        return row['uuid'] + '_' + str(row["i_sz"]+1)
all_onsets_df['uuid_extended'] = all_onsets_df.apply(append_uuid, axis=1)

In [ ]:
g = all_onsets_df.groupby("mouse_id")
for group in g:
    print(group[0])
    g2 = group[1].groupby("uuid")
    for grp in g2:
        print("\t" + grp[0])
        print("\t" + str(len(grp[1])) + "\n")

`all_onsets_df` now contains each recording with seizure and/or SD each neuron. For each neuron, there is a value for onset of each SD and seizure wave (NaN for all neurons in a session if none occurred in the recording) 

In [ ]:
# todo: check last missing recording for T333. Try matching neurons per recording?

## Observe quartiles in one session, one event, per neuron

In [ ]:
MOUSE_ID="T352"

In [ ]:
uuids = all_onsets_df["uuid_extended"].unique()
selected_uuid = uuids[3]
selected_df = all_onsets_df[all_onsets_df["uuid"] == selected_uuid]
event_type = "quantile_sz"#"quantile1"

fig = plt.figure(figsize=(18,18), )
plt.suptitle(f"Top={ddoc.getTopDirection(MOUSE_ID)}", fontsize=24)
ax = plt.gca()
sns.scatterplot(x="x", y="y",
                hue=event_type, style=event_type, size=event_type,
                sizes=(100,400),
                data=selected_df, ax=ax,)




ax.set_xlim((0, 512))
ax.set_ylim((0,512))
ax.legend(fontsize=20)
ax.invert_yaxis()

plt.show()

## Get quantiles of recruitment

### Average coordinates per quartile per session (per mouse)

In [ ]:
quantile_per_session = all_onsets_df.groupby(["mouse_id", "uuid_extended", "quantile1"], as_index=False)  # NaN quantile_sz values already dropped
q_sd1_df = quantile_per_session.mean()
q_sd1_df["quantile_type"] = "sd1"
q_sd1_df.rename({"quantile1": "quantile"}, axis="columns", inplace=True)
print(q_sd1_df.columns)

quantile_per_session = all_onsets_df.groupby(["mouse_id", "uuid_extended", "quantile2"], as_index=False)  # NaN quantile_sz values already dropped
q_sd2_df = quantile_per_session.mean()
q_sd2_df["quantile_type"] = "sd2"
q_sd2_df.rename({"quantile2": "quantile"}, axis="columns", inplace=True)


quantile_per_session = all_onsets_df.groupby(["mouse_id", "uuid_extended", "quantile_sz"], as_index=False)
q_sz_df = quantile_per_session.mean()
q_sz_df["quantile_type"] = "sz"
q_sz_df.rename({"quantile_sz": "quantile"}, axis="columns", inplace=True)


quantiles_df = pd.concat([q_sd1_df, q_sd2_df, q_sz_df])

### Inspect centre point of quantiles per mouse and per event type

In [ ]:
MOUSE_ID = "T352"
EVENT_TYPE = "sd2"

fig = plt.figure(figsize=(18,18), )
plt.suptitle(f"Top={ddoc.getTopDirection(MOUSE_ID)}", fontsize=24)
ax = plt.gca()
sns.scatterplot(x="x", y="y",
                hue="quantile", style="quantile", size="quantile",
                sizes=(100,400),
                data=quantiles_df[(quantiles_df["mouse_id"] == MOUSE_ID) & (quantiles_df["quantile_type"] == EVENT_TYPE)], ax=ax,)




ax.set_xlim((0, 512))
ax.set_ylim((0,512))
ax.legend(fontsize=20)
ax.invert_yaxis()

plt.show()

In [ ]:
# TODO: for each quantile, plot cells, k-means clustering? This way, can figure out if there might be more than two clusters.

# Vectorize directions

## Get vectorial difference of last minus first quantiles

In [ ]:
# Move from nd2/video-style coordinate system (top left = (0, 0)) to usual plotting coordinate style (bottom left = (0, 0))
quantiles_df["y_mirrored"]= quantiles_df.apply(lambda row: -1*row["y"], axis=1)

In [ ]:
def get_dx(df, colname="x"):
    max_quantile = df["quantile"].max()
    min_quantile = df["quantile"].min()
    x1 = df[df["quantile"] == max_quantile][colname].values[0]
    x0 = df[df["quantile"] == min_quantile][colname].values[0]
    return x1-x0

# add dx and dy columns by finding for each session, each event (sd1, sd2, sz), the average x coordinate of each first and last quantiles, and producing the difference.
qdf = quantiles_df.groupby(["uuid_extended", "quantile_type"], as_index=False).apply(lambda group_df: group_df.assign(dx=lambda gdf: get_dx(gdf, "x"), dy=lambda gdf: get_dx(gdf, "y_mirrored")))#[["mouse_id", "uuid","dx", "dy", "quantile_type"]]

In [ ]:
qdf[qdf["uuid_extended"] == "06ebcf354f5c41519669f187e16de364"]

## Reshape dataframe
Due to the way the vectors were calculated, each quantile per session has the same dx and dy values. Throw away duplicates.

In [ ]:
qdf = qdf[qdf["quantile"] == 0].drop(labels=["quantile", "x", "y"], axis=1)

## Add polar coordinates

In [ ]:
import math

In [ ]:
sign_change_dict = {"bottom": -1, "top": 1}  # for contralateral injections, injection is always on the medial side of the window
qdf["r"] = qdf.apply(lambda row: math.sqrt(math.pow(row["dx"], 2) + math.pow(row["dy"], 2)), axis=1)
qdf["theta"] = qdf.apply(lambda row: math.atan2(row["dy"], row["dx"]), axis=1)
# correct angle s.t. top direction is always towards injection
qdf["theta_inj_top"] = qdf.apply(lambda row: sign_change_dict[ddoc.getInjectionDirection(row["mouse_id"])]*row["theta"], axis=1)

In [ ]:
# TODO: adjust directions by getting mouse window size...

In [ ]:
sns.set(font_scale=1.5) 
sns.set_theme(style="whitegrid")
# Set up a grid of axes with a polar projection
g = sns.FacetGrid(qdf, col="mouse_id", hue="quantile_type",
                  subplot_kws=dict(projection='polar'), aspect=1, height=7,
                  sharex=True, sharey=True, despine=False)

# Draw a scatterplot onto each axes in the grid
g.map(sns.scatterplot, "theta_inj_top", "r", s=200, alpha=0.7)
plt.gca().set_rlim(0, 350)

g.add_legend(fontsize=20)
plt.show()

## Create sd-sz vectors

### Handle seizures that were split in two (sz onset in one session, sd waves in other)

In [ ]:
qdf["uuid_matched"] = qdf["uuid_extended"]
# following two recordings contain 1 seizure-sd event
qdf["uuid_matched"] = qdf["uuid_matched"].replace("65bff16a4cf04930a5cb14f489a8f99b", "30dc55d1a5dc4b0286d132e72f208ca6")
# following recordings do not have sz
qdf = qdf[qdf["uuid_matched"] != "171693d0988c458a96c8198c7b8cfc28"]

In [ ]:
angles_dict = {"mouse_id": [], "uuid_matched": [], "quantile_type": [], "angle_with_sz": [], "cos_angle_with_sz": []}
angles_dict_v2 = {"mouse_id": [], "uuid_matched": [], "angle_type": [], "angle": [], "cos_angle": []} # keep backwards compatibility, but angle_with_sz is not flexible for example if want to look at sd1-sd2 angle

session_groups = qdf.groupby("uuid_matched")
for uuid, group in session_groups:
    sz_row = group[group["quantile_type"] == "sz"]
    sd1_row = group[group["quantile_type"] == "sd1"]
    sd2_row = group[group["quantile_type"] == "sd2"]
    
    sz_len = sz_row["r"].values[0] if len(sz_row["r"].values) > 0 else np.nan
    sd1_len = sd1_row["r"].values[0] if len(sd1_row["r"].values) > 0 else np.nan
    sd2_len = sd2_row["r"].values[0] if len(sd2_row["r"].values) > 0 else np.nan

    
    # u.v = |u|*|v|*cos(theta)
    if len(sz_row["dx"].values) > 0 and len(sd1_row["dx"].values) > 0:
        sz_dot_sd1 = sz_row["dx"].values[0] * sd1_row["dx"].values[0] + sz_row["dy"].values[0] * sd1_row["dy"].values[0]
        costheta1 = sz_dot_sd1/(sz_len*sd1_len)
    else:
        sz_dot_sd1 = np.nan
        costheta1 = np.nan
    if len(sd2_row["dx"].values) > 0 and len(sd1_row["dx"].values) > 0: 
        sz_dot_sd2 = sz_row["dx"].values[0] * sd2_row["dx"].values[0] + sz_row["dy"].values[0] * sd2_row["dy"].values[0]
        costheta2 = sz_dot_sd2/(sz_len*sd2_len)
    else:
        sz_dot_sd2 = np.nan
        costheta2 = np.nan
    
    # add sd1-sd2 angle too
    if len(sd1_row["dx"].values) > 0 and len(sd2_row["dx"].values) > 0:
        sd1_dot_sd2 = sd1_row["dx"].values[0] * sd2_row["dx"].values[0] + sd1_row["dy"].values[0] * sd2_row["dy"].values[0]
        costheta12 = sd1_dot_sd2/(sd1_len*sd2_len)
        
    angles_dict["mouse_id"].append(sz_row["mouse_id"].values[0])
    angles_dict["uuid_matched"].append(uuid)
    angles_dict["quantile_type"].append("sd1")
    angles_dict["angle_with_sz"].append(math.acos(costheta1))
    angles_dict["cos_angle_with_sz"].append(costheta1)
    
    angles_dict["mouse_id"].append(sz_row["mouse_id"].values[0])
    angles_dict["uuid_matched"].append(uuid)
    angles_dict["quantile_type"].append("sd2")
    angles_dict["angle_with_sz"].append(math.acos(costheta2))
    angles_dict["cos_angle_with_sz"].append(costheta2)
    
    # sz-sd1
    angles_dict_v2["mouse_id"].append(sz_row["mouse_id"].values[0])
    angles_dict_v2["uuid_matched"].append(uuid)
    angles_dict_v2["angle_type"].append("sz-sd1")
    angles_dict_v2["angle"].append(math.acos(costheta1))
    angles_dict_v2["cos_angle"].append(costheta1)
    # sz-sd2
    angles_dict_v2["mouse_id"].append(sz_row["mouse_id"].values[0])
    angles_dict_v2["uuid_matched"].append(uuid)
    angles_dict_v2["angle_type"].append("sz-sd2")
    angles_dict_v2["angle"].append(math.acos(costheta2))
    angles_dict_v2["cos_angle"].append(costheta2)    
    # sd1-sd2
    angles_dict_v2["mouse_id"].append(sz_row["mouse_id"].values[0])
    angles_dict_v2["uuid_matched"].append(uuid)
    angles_dict_v2["angle_type"].append("sd1-sd2")
    angles_dict_v2["angle"].append(math.acos(costheta12))
    angles_dict_v2["cos_angle"].append(costheta12) 

In [ ]:
df_angles = pd.DataFrame(angles_dict)

In [ ]:
df_angles.pivot_table(index="mouse_id",columns='r_dummy', values="angle_with_sz")

In [ ]:
r_map = {"sd1": 1, "sd2": 2}
df_angles["r_dummy"] = df_angles.apply(lambda row: r_map[row["quantile_type"]], axis=1)

In [ ]:
sns.set(font_scale=1.5) 
sns.set_theme(style="whitegrid")
# Set up a grid of axes with a polar projection
g = sns.FacetGrid(df_angles, col="mouse_id", hue="quantile_type",
                  subplot_kws=dict(projection='polar'), aspect=1, height=7,
                  sharex=True, sharey=True, despine=False)

# Draw a scatterplot onto each axes in the grid
g.map(sns.scatterplot, "angle_with_sz", "r_dummy", s=400, alpha=0.5)
plt.gca().set_rlim(0, 2.25)


g.add_legend(fontsize=20)
plt.show()

In [ ]:
ddoc.getInjectionDirection("T352")

In [ ]:
ddoc.getMouseWinInjInfo("T301")

In [ ]:
ddoc.getMouseWinInjInfo("T329")

In [ ]:
ddoc.getTopDirection("T301")

In [ ]:
df_angles.to_excel("angles.xlsx")

In [ ]:
qdf.to_excel("quantiles.xlsx")

In [ ]:
sns.set(font_scale=1.5) 
sns.set_theme(style="whitegrid")
# Set up a grid of axes with a polar projection
g = sns.FacetGrid(qdf, col="mouse_id", hue="quantile_type",
                  subplot_kws=dict(projection='polar'), aspect=1, height=7,
                  sharex=True, sharey=True, despine=False)

# Draw a scatterplot onto each axes in the grid
g.map(sns.scatterplot, "theta", "r", s=200, alpha=0.7)
plt.gca().set_rlim(0, 350)

g.add_legend(fontsize=20)
plt.show()

## Set plotting details
Color scheme, arrow widths, order of event types (sz, sd1, sd2)...

In [ ]:
# create colors with hues
rgbs = []
for color in mcolors.TABLEAU_COLORS.values():
    rgbs.append(color)

In [ ]:
#this color scheme does not work properly. Need to adjust v values...

# create 2d array: [color][lightness_variant]
def getColorScheme(n_shades=4):
    color_scheme = []
    for color in mcolors.TABLEAU_COLORS.values():
        h, s, v = mcolors.rgb_to_hsv(mcolors.to_rgb(color))
        vs = np.linspace(v, 1, n_shades+1)  # first color in linspace is #000000, which will not be used
        color_row = [mcolors.to_hex(mcolors.hsv_to_rgb((h, s, v_i))) for v_i in vs[1:]]
        color_scheme.append(color_row)
    return color_scheme
#color_scheme = getColorScheme(4)
#mcolors.TABLEAU_COLORS with tints, [hue][dark to light]
color_scheme = [
    ['#1f77b4', "#258fd8", "#49a2e0", "#6db5e6", "#92c7ec", "#b6daf2"],
    ["#ff7f0e", "#ff9130", "#ffa453", "#ffb675", "#ffc898", "#ffdaba"],
    ["#2ca02c", "#35c235", "#54d054", "#76d976", "#98e398", "#baecba"],
    ["#d62728", "#dd4546", "#e36464", "#e88383", "#eea2a2", "#f4c1c1"],
    ["#9467bd", "#a37dc6", "#b392d0", "#c2a8d9", "#d1bee3", "#e0d4ec"],
    ["#8c564b", "#a7675a", "#b58176", "#c49a91", "#d3b3ad", "#e2ccc8"],
    ["#e377c2", "#e78acb", "#eb9ed3", "#efb1dc", "#f3c5e5", "#f7d8ee"]
]

In [ ]:
# assign an index to each mouse. 0.0, 1.0, 2.0, ...
qdf["mouse_index"] = qdf["mouse_id"].rank(method='dense') - 1
qdf["mouse_index"] = qdf["mouse_index"].apply(int)

In [ ]:
# assert enough colors are available
assert len(color_scheme) > qdf["mouse_index"].max()

In [ ]:
#event_types = list(qdf.quantile_type.unique())
event_types = ["sd2", "sd1", "sz"]  # order for vector length

In [ ]:
ARROW_WIDTHS = {"sd2": 12, "sd1": 6, "sz": 2}
ARROW_HEAD = 16

## Create arrow plots

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')

# plot arrows
for i, row in qdf.iterrows():
    r = event_types.index(row["quantile_type"])+1# row['r']
    theta = row['theta_inj_top']
    ax.annotate('', xy=(theta, r), xytext=(0, 0),
                arrowprops=dict(facecolor=color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index(row["quantile_type"])], edgecolor='none', width=ARROW_WIDTHS[row["quantile_type"]], headwidth=ARROW_HEAD, alpha=0.9))

# set the radial limits
max_r = len(qdf["quantile_type"].unique())-1#qdf['r'].max()
ax.set_ylim(0, max_r + 1)
ax.set_rgrids(np.linspace(0, max_r+1, num=len(event_types)+1))
ax.set_yticklabels([""] + event_types)
ax.set_xticklabels(['', '', 'medial/injection', '', '', '', 'lateral', ''], fontsize=20)

#ax.set_xticklabels(event_types) 
# display the plot
plt.show()

In [ ]:
# plot mean directions per mouse, per event type

#grouped_id_etype = qdf.groupby(["mouse_id", "quantile_type"], as_index=False).apply()
#for i_group, group in grouped_id_etype:
#    group["x_"]
#    print(group["r"].mean())

In [ ]:
#qdf = quantiles_df.groupby(["uuid_extended", "quantile_type"], as_index=False).apply(lambda group_df: group_df.assign(dx=lambda gdf: get_dx(gdf, "x"), dy=lambda gdf: get_dx(gdf, "y_mirrored")))
df_mean_vectors_per_mouse = qdf.groupby(['mouse_id', 'quantile_type'], as_index=False).mean().drop(columns=["y_mirrored", "r", "theta", "theta_inj_top"])

### Add aligned angle (where top is injection site direction)

In [ ]:
sign_change_dict = {"bottom": -1, "top": 1}  # for contralateral injections, injection is always on the medial side of the window
df_mean_vectors_per_mouse["r"] = df_mean_vectors_per_mouse.apply(lambda row: math.sqrt(math.pow(row["dx"], 2) + math.pow(row["dy"], 2)), axis=1)
df_mean_vectors_per_mouse["theta"] = df_mean_vectors_per_mouse.apply(lambda row: math.atan2(row["dy"], row["dx"]), axis=1)
df_mean_vectors_per_mouse["theta_inj_top"] = df_mean_vectors_per_mouse.apply(lambda row: sign_change_dict[ddoc.getInjectionDirection(row["mouse_id"])]*row["theta"], axis=1)

In [ ]:
df_mean_vectors_per_mouse

In [ ]:
quantile_angles_pivot = df_mean_vectors_per_mouse.pivot_table(index="mouse_id", columns='quantile_type', values='theta_inj_top')
quantile_angles_pivot["sz-sd1"] = (quantile_angles_pivot["sz"] - quantile_angles_pivot["sd1"] + math.pi) % (2 * math.pi) - math.pi # convert to (-pi, pi)
quantile_angles_pivot["sz-sd2"] = (quantile_angles_pivot["sz"] - quantile_angles_pivot["sd2"] + math.pi) % (2 * math.pi) - math.pi # convert to (-pi, pi)

In [ ]:
def radianTo02PiRange(angle_rad):
    return (angle_rad)%(2*math.pi)


def getSectorsForMouse(mouse_id, mouse_index):
    theta_sz = radianTo02PiRange(quantile_angles_pivot.loc[mouse_id]["sz"])
    theta_sd1 = radianTo02PiRange(quantile_angles_pivot.loc[mouse_id]["sd1"])
    theta_sd2 = radianTo02PiRange(quantile_angles_pivot.loc[mouse_id]["sd2"])

    # width angle: total angular region covered.
    # 1. find smaller angle between them
    theta1_min = min(theta_sz, theta_sd1)
    theta1_max = max(theta_sz, theta_sd1)
    theta2_min = min(theta_sz, theta_sd2)
    theta2_max = max(theta_sz, theta_sd2)

    dtheta_1 = theta1_max - theta1_min
    theta1_width = min(dtheta_1, 2*math.pi - dtheta_1)
    dtheta_2 = theta2_max - theta2_min
    theta2_width = min(dtheta_2, 2*math.pi - dtheta_2)

    # mid angle: between the two arrows.
    # 1. find larger angle - smaller angle difference; if > pi, should take counter-clockwise difference.
    if dtheta_1 <= math.pi:  # can take angle as-is
        theta1_mid = (theta_sz + theta_sd1)/2 #np.deg2rad((theta1 + theta2)/2)
    else:
        # the middle angle clockwise needs to be mirrored around the origin. Done by subtracting pi.
        theta1_mid = (theta_sz + theta_sd1)/2 - math.pi
    if dtheta_2 <= math.pi:
        theta2_mid = (theta_sz + theta_sd2)/2
    else:
        theta2_mid = (theta_sz + theta_sd2)/2 - math.pi
    #ax.bar(x=theta1_mid, height = 20, width=theta1_width, bottom=r0, color=color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index(row["quantile_type"])])
    #r0 += 20
    #
    #r0 += 20

    # TODO: make sure sd1 and sd2 exist

    # get vector lengths for radii of sectors
    r0 = 10  # the inner perimeter of each segment will be at 10.
    r1 = df_mean_vectors_per_mouse[(df_mean_vectors_per_mouse["mouse_id"] == mouse_id) & (df_mean_vectors_per_mouse["quantile_type"] == "sd1")]["r"].values[0]
    r2 = df_mean_vectors_per_mouse[(df_mean_vectors_per_mouse["mouse_id"] == mouse_id) & (df_mean_vectors_per_mouse["quantile_type"] == "sd2")]["r"].values[0]
    rsz = df_mean_vectors_per_mouse[(df_mean_vectors_per_mouse["mouse_id"] == mouse_id) & (df_mean_vectors_per_mouse["quantile_type"] == "sz")]["r"].values[0]

    r_padding = 6  # to make arrows visible, leave this much space between arrow point and sector perimeter
    rmax_sd1 = min(r1-r_padding, rsz-r_padding)
    rmax_sd2 = min(r2-r_padding, rsz-r_padding)
    assert rmax_sd1 > r0
    assert rmax_sd2 > r0


    # avoid same-size sectors for same mouse
    if rmax_sd1 == rmax_sd2:
        rmax_sd2 -= 20

    # calculate inner perimeter (bottom) and height for both sectors
    height1 = rmax_sd1 - r0 - r_padding
    height2 = rmax_sd2 - r0 - r_padding
    bottom1 = r0
    bottom2 = r0


    # get colors
    color1 = color_scheme[int(mouse_index)][len(event_types) - event_types.index("sd1")]
    color2 = color_scheme[int(mouse_index)][len(event_types) - event_types.index("sd2")]
    return [[theta1_mid, height1, theta1_width, bottom1, color1], [theta2_mid, height2, theta2_width, bottom2, color2]]

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')
# contains all arc properties gathered for later plotting:
# mice_list = [<mouse_id1>, <mouse_id1>, <mouse_id2>, <mouse_id2>, ...]
# for each mouse_id entry, there is one array in arcs_list:
# [theta1_mid, height1, theta1_width, bottom1, color1], [theta2_mid, height2, theta2_width, bottom2, color2]
# it is a list to be able to sort the plotting
# TODO: create this dictionary, then loop through keys, loop through keys inside, and if exists, plot the corresponding arc.
# TODO: change arc colors to shades of sd1 and sd2
# TODO: change arcs to
mice_list = []
arcs_list = [] 



# plot arrows
for i, row in df_mean_vectors_per_mouse.iterrows():
    r = row['r']#event_types.index(row["quantile_type"])+1# 
    theta = row['theta_inj_top']
    ax.annotate('', xy=(theta, r), xytext=(0, 0),
                arrowprops=dict(facecolor=color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index(row["quantile_type"])], edgecolor='grey', width=ARROW_WIDTHS[row["quantile_type"]], headwidth=ARROW_HEAD, alpha=0.9))
    if row["mouse_id"] not in mice_list:  # avoid adding sectors for the same mouse again
        sectors = getSectorsForMouse(row["mouse_id"], row["mouse_index"])
        
        # add data to list
        mice_list.append(row["mouse_id"])
        arcs_list.append(sectors[0]) #[row["mouse_id"]] = arc_data
        mice_list.append(row["mouse_id"])
        arcs_list.append(sectors[1])
        
        

# sort by descending outer perimeter so all of them visible by plotting smaller on top of larger sectors
for sector in sorted(arcs_list, key=lambda row: row[1], reverse=True):
    ax.bar(x=sector[0], height=sector[1], width=sector[2], bottom=sector[3], color=sector[4], edgecolor="grey")

        
#r = Rectangle((0., 0.), 100., 100, color="red")
#p = PatchCollection([Rectangle((np.deg2rad(20), 100), 45, 50, color='blue')])
#ax.add_collection(p)

# set the radial limits
max_r = df_mean_vectors_per_mouse['r'].max() #len(qdf["quantile_type"].unique())-1#
ax.set_ylim(0, max_r + 1)
#ax.set_rgrids(np.linspace(0, max_r+1, num=len(event_types)+1))
#ax.set_yticklabels([""] + event_types)
ax.set_xticklabels(['', '', 'medial/injection', '', '', '', 'lateral', ''], fontsize=20)  # top direction should be "medial/injection"

#ax.set_xticklabels(event_types) 
# display the plot
plt.show()

In [ ]:
# TODO: length of vectors should still somehow correspond to average... If length is small, the direction might be insignificant

In [ ]:
# TODO: use circular heatmap or roseplot!

In [ ]:
# TODO: mark the angles between average vectors (sz-sd1, sz-sd2) for each mouse

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')
r0 = 0.1
r_out_sd1 = event_types.index("sd1") + 0.8  # 0-indexing + 1 - 0.2 to make arrow points still visible
r_out_sd2 = event_types.index("sd2") + 0.8

# contains all arc properties gathered for later plotting:
# mice_list = [<mouse_id1>, <mouse_id1>, <mouse_id2>, <mouse_id2>, ...]
# for each mouse_id entry, there is one array in arcs_list:
# [theta1_mid, height1, theta1_width, bottom1, color1], [theta2_mid, height2, theta2_width, bottom2, color2]
# it is a list to be able to sort the plotting
# TODO: create this dictionary, then loop through keys, loop through keys inside, and if exists, plot the corresponding arc.
# TODO: change arc colors to shades of sd1 and sd2
# TODO: change arcs to
mice_list = []
arcs_list = [] 


# to also plot the relative amplitudes of each vector, keep track of [theta, r] pairs. Later, plot normalized r (normalized to 3)
length_markers_list = [[],[],[]]
r_max = df_mean_vectors_per_mouse['r'].max()  # use r_max to normalize to 3. Longest vector will reach 3...


def radianTo02PiRange(angle_rad):
    return (angle_rad)%(2*math.pi)

outline_color = "none"  # "grey" 

# plot arrows
for i, row in df_mean_vectors_per_mouse.iterrows():
    r = event_types.index(row["quantile_type"])+1 # length depends on type (sz, sd1, sd2)
    theta = row['theta_inj_top']
    r_original = row['r']
    # add theta and length to points list
    length_markers_list[0].append(theta)
    length_markers_list[1].append((r_original/r_max)*len(event_types))
    length_markers_list[2].append(color_scheme[int(row["mouse_index"])][0])
    
    ax.annotate('', xy=(theta, r), xytext=(0, 0),
                arrowprops=dict(facecolor=color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index(row["quantile_type"])], edgecolor=outline_color, width=ARROW_WIDTHS[row["quantile_type"]], headwidth=ARROW_HEAD, alpha=0.9))
    
    
    if row["mouse_id"] not in mice_list:
        theta_sz = radianTo02PiRange(quantile_angles_pivot.loc[row["mouse_id"]]["sz"])
        theta_sd1 = radianTo02PiRange(quantile_angles_pivot.loc[row["mouse_id"]]["sd1"])
        theta_sd2 = radianTo02PiRange(quantile_angles_pivot.loc[row["mouse_id"]]["sd2"])
        
        # width angle: total angular region covered.
        # 1. find smaller angle between them
        theta1_min = min(theta_sz, theta_sd1)
        theta1_max = max(theta_sz, theta_sd1)
        theta2_min = min(theta_sz, theta_sd2)
        theta2_max = max(theta_sz, theta_sd2)
        
        dtheta_1 = theta1_max - theta1_min
        theta1_width = min(dtheta_1, 2*math.pi - dtheta_1)
        dtheta_2 = theta2_max - theta2_min
        theta2_width = min(dtheta_2, 2*math.pi - dtheta_2)
        
        # mid angle: between the two arrows.
        # 1. find larger angle - smaller angle difference; if > pi, should take counter-clockwise difference.
        if theta1_max - theta1_min <= math.pi:  # can take angle as-is
            theta1_mid = (theta_sz + theta_sd1)/2 #np.deg2rad((theta1 + theta2)/2)
        else:
            # the middle angle clockwise needs to be mirrored around the origin. Done by subtracting pi.
            theta1_mid = (theta_sz + theta_sd1)/2 - math.pi
        if theta2_max - theta2_min <= math.pi:
            theta2_mid = (theta_sz + theta_sd2)/2
        else:
            theta2_mid = (theta_sz + theta_sd2)/2 - math.pi

        
        
        
        
        # calculate inner perimeter (bottom) and height for both sectors
        height1 = r_out_sd1 - r0
        height2 = r_out_sd2 - r0
        bottom1 = r0
        bottom2 = r0
        
        
        # get colors
        color1 = color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index("sd1")]
        color2 = color_scheme[int(row["mouse_index"])][len(event_types) - event_types.index("sd2")]
        
        # add data to list
        mice_list.append(row["mouse_id"])
        arcs_list.append([theta1_mid, height1, theta1_width, bottom1, color1]) #[row["mouse_id"]] = arc_data
        mice_list.append(row["mouse_id"])
        arcs_list.append([theta2_mid, height2, theta2_width, bottom2, color2])
        
        r_out_sd1 -= 0.2
        r_out_sd2 -= 0.2
        
        

# sort by descending outer perimeter so all of them visible by plotting smaller on top of larger sectors
for sector in sorted(arcs_list, key=lambda row: row[1], reverse=True):
    ax.bar(x=sector[0], height=sector[1], width=sector[2], bottom=sector[3], color=sector[4], edgecolor=outline_color)


#r = Rectangle((0., 0.), 100., 100, color="red")
#p = PatchCollection([Rectangle((np.deg2rad(20), 100), 45, 50, color='blue')])
#ax.add_collection(p)

# set the radial limits
max_r = len(qdf["quantile_type"].unique())-1#
plot_length = False
if plot_length:  # add scatter points for relative lengths
    for i_length_marker in range(len(length_markers_list[0])):
        theta = length_markers_list[0][i_length_marker]
        r = length_markers_list[1][i_length_marker]
        color = length_markers_list[2][i_length_marker]
        ax.scatter(theta, r, color=color, s=40)# color="black", s=20)
    
ax.set_ylim(0, max_r + 1)
ax.set_rgrids(np.linspace(0, max_r+1, num=len(event_types)+1))
ax.set_yticklabels([""] + event_types)
ax.set_xticklabels(['', '', 'medial/injection', '', '', '', 'lateral', ''], fontsize=20)  # top direction should be "medial/injection"

    
# display the plot
plt.show()

### Get sectors for all recordings individually

In [ ]:
pt = qdf.pivot_table(index=["uuid_matched", "mouse_id", "mouse_index"], columns=["quantile_type"], values=["theta_inj_top", "r"]).sort_values(by=["mouse_id"])
pt.loc["06ebcf354f5c41519669f187e16de364"].index.values[0]

In [ ]:
pt

In [ ]:
def getSectorAnglesForVectors(theta1, theta2, force_direction=False, direction="ccw"):
    # direction: cw for clockwise, ccw for counter-clockwise
    dtheta = abs(theta1 - theta2)
    if force_direction:
        if direction == "ccw":
            # to make only counter-clockwise sectors (from theta1 to theta2):
            #  0. the width does not change, no matter in what interval the two angles are defined
            theta_width = (theta2 - theta1)%(2*math.pi)
            #  1. bring them to a form where theta2 > theta1
            theta2_adjusted = (theta2)%(2*math.pi)
            theta1_adjusted = (theta1)%(2*math.pi)
            if theta2_adjusted < theta1_adjusted:  # need to swap to (-pi, pi) range
                theta2_adjusted = (theta2_adjusted - math.pi)%(2*math.pi)
                theta1_adjusted = (theta1_adjusted - math.pi)%(2*math.pi)
            theta_mid= ((theta1 + theta2)/2)%(2*math.pi)
        elif direction == "cw":
            # clockwise sectors from theta1 to theta2. This means the signs are reversed
            # bring the angles to a range where theta2 < theta1. Try (0, 2pi) and (-pi, pi)
            theta2_adjusted = (theta2)%(2*math.pi)
            theta1_adjusted = (theta1)%(2*math.pi)
            if theta2_adjusted > theta1_adjusted:  # need to swap to (-pi, pi) range
                theta2_adjusted = (theta2_adjusted - math.pi)%(2*math.pi)
                theta1_adjusted = (theta1_adjusted - math.pi)%(2*math.pi)
            theta_width = (theta1_adjusted - theta2_adjusted)%(2*math.pi)
            theta_mid= ((theta1_adjusted + theta2_adjusted)/2)%(2*math.pi)
    else:  # find smaller angle for sectors
        theta_width = min(dtheta, 2*math.pi - dtheta)
        if dtheta <= math.pi:  # can take angle as-is
            theta_mid= (theta1 + theta2)/2 #np.deg2rad((theta1 + theta2)/2)
        else:
            # the middle angle clockwise needs to be mirrored around the origin. Done by subtracting pi.
            theta_mid = (theta1 + theta2)/2 - math.pi
    return (theta_mid, theta_width)



In [ ]:
sectors = []  # list of [theta_mid, height, theta_width, bottom, color]
seizure_indicators = []  # list of (theta, r_max = height + r_0, color)
r_0 = 0.2
r_top = 0.4
for indices, row in pt.iterrows(): # (uuid_matched, mouse_id, mouse_index), df[theta_inj_top/r][sz/sd1/sd2]
    # get one or two sectors 
    # get sd1
    theta_mid, theta_width = getSectorAnglesForVectors(row["theta_inj_top"]["sz"], row["theta_inj_top"]["sd1"], False, "ccw")
    color = color_scheme[indices[2]][len(event_types) - event_types.index("sd1")]
    height = r_top - r_0
    sectors.append([theta_mid, height, theta_width, r_0, color])
    r_top += 0.2    
    # get sd2 (if exists)
    if not np.isnan(row["theta_inj_top"]["sd2"]):
        theta_mid, theta_width = getSectorAnglesForVectors(row["theta_inj_top"]["sz"], row["theta_inj_top"]["sd2"], False, "ccw")
        color = color_scheme[indices[2]][len(event_types) - event_types.index("sd2")]
        height = r_top - r_0
        sectors.append([theta_mid, height, theta_width, r_0, color])
    # get seizure direction for showing as line
    color = color_scheme[indices[2]][len(event_types) - event_types.index("sz")]
    seizure_indicators.append((row["theta_inj_top"]["sz"], r_top, color))
    r_top += 0.4
    


fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')

# sort by descending outer perimeter so all of them visible by plotting smaller on top of larger sectors
i_sector = len(sectors) -1
while i_sector > 0:
    sector = sectors[i_sector]
    ax.bar(x=sector[0], height=sector[1], width=sector[2], bottom=sector[3], color=sector[4], edgecolor="grey")
    i_sector -= 1
# seizure direction
for sz_ind in seizure_indicators:
    ax.annotate('', xy=(sz_ind[0], sz_ind[1]), xytext=(0, 0),
            arrowprops=dict(facecolor="black", edgecolor='none', width=1, headwidth=4, alpha=0.9))
# set the radial limits
max_r = sectors[-1][1] + sectors[-1][3]

ax.set_ylim(0, max_r)
#ax.set_rgrids(np.linspace(0, max_r+1, num=len(event_types)+1))
#ax.set_yticklabels([""] + event_types)
ax.set_xticklabels(['', '', 'medial/injection', '', '', '', 'lateral', ''], fontsize=20)  # top direction should be "medial/injection"

    
# display the plot
plt.show()

# Change seaborn parameters

In [ ]:
sns.set(font_scale=1.5)

## Bar plot angles

In [ ]:
df_angles_v2 = pd.DataFrame(angles_dict_v2)  # this dict contains sd1-sd2 angles too
df_angles_v2["angle_deg"] = df_angles_v2["angle"].apply(lambda rad: 360.*rad/(2*math.pi))

In [ ]:
fig = plt.figure(figsize=(12,8))
sns.barplot(data=df_angles_v2, x="mouse_id", y="angle_deg", hue="angle_type", errorbar="sd")
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,8))
sns.barplot(data=df_angles_v2, x="angle_type", y="angle_deg", errorbar="sd")
plt.show()

In [ ]:
df_angles_v2["angle_deg"]

# Speed analysis

## Seizure speed

In [ ]:
#all_onsets_df[all_onsets_df["uuid"] == "171693d0988c458a96c8198c7b8cfc28"]

In [ ]:
onset_df_1 = pd.read_hdf("D:\\Analysis_v1\\T352\\thy6s_1.081219.1339_23-04-13_19-14-15_grid.h5")

In [ ]:
comp_df = all_onsets_df[all_onsets_df["uuid"] == "79fb974821f34e3abdcf5ca650e1c0f4"]

onset_df_1.columns

In [ ]:
# ['neuron_id', 'x', 'y', 'row', 'col', 'tile', 'onset1', 'onset2',
#       'quantile1', 'quantile2', 'onset_sz', 'quantile_sz']
comp_df[['neuron_id', 'row','col','tile']].describe()

In [ ]:
comp_df.drop(columns=['i_sz', 'mouse_id', 'uuid', 'uuid_extended']).columns#.describe()

In [ ]:
onset_df_1.describe()

In [ ]:
# get all grids. Shape = (n_events, *grid_shape)
all_grids = np.zeros(shape=(3*len(all_onsets_df["uuid_extended"].unique()), *grid_shape), dtype=np.float64)
uuids = []

In [ ]:
for uuid_ext, session in all_onsets_df.groupby("uuid_extended"):
    events_in_session = session.groupby("tile").median().pivot_table(index="row",columns="col",values=["onset1", "onset2", "onset_sz"])
    

In [ ]:
for i_group, group in all_onsets_df.groupby("uuid_extended"):
    group.groupby("tile").median().pivot_table(index="row",columns="col",values="x")

In [ ]:
all_onsets_df.columns

In [ ]:
onset_df_1.groupby("tile").median().pivot_table(index="row",columns="col",values=["onset1", "onset2", "onset_sz"])

In [ ]:
all_onsets_df.groupby("tile").median()#.pivot_table(index="row",columns="col",values="onset_sz")

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
plt.suptitle(f"Seizure {i_sz+1}")
sns.heatmap(onset_df_1.groupby("tile").median().pivot_table(index="row",columns="col",values="onset1"), annot=False, linewidths=.5, ax=ax)
#ax.invert_yaxis()
#ax.invert_xaxis()
plt.show()

In [ ]:
a = comp_df["x"].reset_index()

In [ ]:
comp_df = comp_df.reset_index()

In [ ]:
comp_df

In [ ]:
comp_df["x"] == onset_df_1["x"]

In [ ]:
comp_df

In [ ]:
comp_df.describe(include='all')

In [ ]:
all_onsets_df.info()

In [ ]:
onset_df_1.info()